In [2]:
# Basic data and viz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Dash stuff
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input



In [22]:
# Reading in data
data = pd.read_csv("msa_merged_data.csv")
data["BEGIN_DATE_GMT"] = pd.to_datetime(data["BEGIN_DATE_GMT"])
data['year'] = data['BEGIN_DATE_GMT'].dt.year
data.set_index("BEGIN_DATE_GMT", drop = False, inplace=True)

In [23]:
data.columns

Index(['BEGIN_DATE_GMT', 'HE', 'POOL_PRICE', 'AIL_DEMAND', 'Avg_temp',
       'Weighted_Avg_Temp', 'Calgary_temp', 'Edmonton_temp', 'FortMM_Temp',
       'Lethbridge_temp', 'future 1', 'future 2', 'future 3', 'future 4',
       'WTI spot', 'dayofweek', 'month', 'year', 'holiday', 'workingday'],
      dtype='object')

In [4]:
month_peaks = data.resample('M', label = 'left')['AIL_DEMAND'].max()
month_avg = data.resample('M', label = 'left')['AIL_DEMAND'].mean()
month_troughs = data.resample('M', label = 'left')['AIL_DEMAND'].min()

month_avg.head()

BEGIN_DATE_GMT
2009-12-31    8545.156038
2010-01-31    8540.630952
2010-02-28    8158.940860
2010-03-31    7773.972222
2010-04-30    7707.354839
Freq: M, Name: AIL_DEMAND, dtype: float64

In [18]:
month_avg = data.resample('M', label = 'right')[['AIL_DEMAND',"month", "year"]].mean()
month_avg.head()

,AIL_DEMAND,month,year
BEGIN_DATE_GMT,,,
2010-01-31,8545.156038,1,2010
2010-02-28,8540.630952,2,2010
2010-03-31,8158.940860,3,2010
2010-04-30,7773.972222,4,2010
2010-05-31,7707.354839,5,2010


In [16]:
#month_avg["year"] = month_avg["year"].astype("string")

AIL_DEMAND    float64
month           int64
year           string
dtype: object

In [21]:
px.scatter(month_avg, x = "month", y = "AIL_DEMAND", facet_col = "year", facet_col_wrap= 5)

In [29]:
daily_avg = data.resample("D").mean()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = np.arange(len(daily_avg[daily_avg["year"] == 2019])),
    y = daily_avg[daily_avg["year"] == 2019]["AIL_DEMAND"],
    name  = "2019"
) )
fig.add_trace(go.Scatter(
    x = np.arange(len(daily_avg[daily_avg["year"] == 2020])),
    y = daily_avg[daily_avg["year"] == 2020]["AIL_DEMAND"],
    name  = "2020"
) )
fig.show()

In [32]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = month_avg[month_avg["year"] == 2020]["month"],
    y = month_avg[month_avg["year"] == 2019]["AIL_DEMAND"],
    name  = "2019"
) )
fig.add_trace(go.Scatter(
    x = month_avg[month_avg["year"] == 2020]["month"],
    y = month_avg[month_avg["year"] == 2020]["AIL_DEMAND"],
    name  = "2020"
) )
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)
fig.show()

In [54]:
daily_avg = data.resample("D", label = "left").mean().rolling("30D").mean()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = np.arange(len(daily_avg[daily_avg["year"] == 2019])),
    y = daily_avg[daily_avg["year"] == 2019]["AIL_DEMAND"],
    name  = "2019"
) )
fig.add_trace(go.Scatter(
    x = np.arange(len(daily_avg[daily_avg["year"] == 2020])),
    y = daily_avg[daily_avg["year"] == 2020]["AIL_DEMAND"],
    name  = "2020"
) )

fig.update_layout(
     xaxis = dict(tickmode = 'array',
        tick0 = 0,
        tickvals = np.arange(1, 365, 30 ),
        ticktext = ['Jan', 'Feb', 'Mar', 'April', 'May', 'June', 'July', "Aug", "Sep","Oct","Nov","Dec"]
        )
)
fig.show()

In [74]:
year_change = data.groupby("year")["AIL_DEMAND"].mean().pct_change() * 100
oil_avg = data.groupby("year")["WTI spot"].mean()
pool_price_avg = data.groupby("year")["POOL_PRICE"].mean()

In [81]:
# Temp vs. Consumption Line Graph
demand_v_oil_line = make_subplots(specs=[[{"secondary_y": True}]])

demand_v_oil_line.add_trace(go.Bar(x= year_change.index,
                                     y= year_change,
                                     name="Average AIL Demand"
                                     ), secondary_y=False)
demand_v_oil_line.add_trace(go.Scatter(x=oil_avg.index,
                                     y= oil_avg,
                                     name="WTI Spot Price"
                                     ), secondary_y=True)

demand_v_oil_line.update_layout(title_text="Annual Average AIL Demand vs. WTI Spot Price from 2011 - 2020")
# Set x-axis title
demand_v_oil_line.update_xaxes(title_text="Time")

# Set y-axes titles
demand_v_oil_line.update_yaxes(
    title_text="AIL Demand (% Change)", secondary_y=False)
demand_v_oil_line.update_yaxes(title_text="WTI Oil Spot ($)", 
                    range=[0, 120],
                    secondary_y=True)

In [94]:
pool_price_avg_line = go.Figure()

pool_price_avg_line.add_trace(go.Scatter(x= pool_price_avg.index,
                                     y= pool_price_avg,
                                     ))

pool_price_avg_line.update_yaxes(range=[0, 90])
pool_price_avg_line.update_layout(shapes=[ dict(
                type="line",
                yref='y1',
                y0= data["WTI spot"].mean(),
                y1= np.mean(data["WTI spot"]),
                xref='x1',
                x0= pool_price_avg.index.min(),
                x1= pool_price_avg.index.max(),
                line=dict(
                    color="Red",
                    width=4,
                    dash="dashdot",
                ) ) ]
        )
pool_price_avg_line.show()                           

In [96]:
oil_avg

year
2010     81.773960
2011     93.838868
2012     94.013470
2013    101.030542
2014    102.484924
2015     61.860134
2016     57.127411
2017     66.026672
2018     84.053736
2019     75.688137
2020     51.873749
Name: WTI spot, dtype: float64